In [ ]:
from scipy import optimize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)

## Total RRs

Remove the existing numbers and add your total move RRs in the array below

In [ ]:
total_RR = np.array([2.58, 1.43, 0.02, 4.35, 9.78, 4.33, 0.08, 0.49, 0.22, 0.23, 26.8, 4, 92, 14.375, 1.7, 
            0.22, 10.87, 0.31, 0.42, 1.2, 3.77, 1.21])

In [ ]:
# calculate RR earned based on partials
def earned_rr(partials):
    p1 = partials[0]
    p1_percent = partials[1]
    p2 = partials[2]
    p2_percent = partials[3]
    
    earned_rr = 0
    
    for rr in total_RR:
        if rr < p1: 
            earned_rr -= 1
        elif rr >= p1 and rr <= p2:
            earned_rr += p1*p1_percent
        elif rr >= p2:
            earned_rr += p2*p1_percent*p2_percent
    
    return earned_rr

# the closer we are to the toal RR possible, the better
def earned_rr_loss(partials):   
    return np.sum(total_RR) - earned_rr(partials)

In [ ]:
# limit partial percentages to equal 1 since we can't take off more than 100% of a position
cons = [
    {'type': 'eq', 'fun': lambda x: x[1] + x[3] - 1 }
]

largest_total_rr = np.max(total_RR)

## First Partial Information

Edit `largest_first_partial` to match the largest first partial you want to take

Edit `first_partial_step` to check every X number between 0 and `largest_first_partial` 

### Note: Setting `first_partial_step` to lower numbers requires longer runtime

In [ ]:
largest_first_partial = 10
first_partial_step = 1
first_partials = list(range(0, largest_first_partial, first_partial_step))

## Second Partial Information

Edit `second_partial_step` to check every X number between your `largest_first_partial` and largest total partial

### Note: Setting `second_partial_step` to lower numbers requires longer runtime

In [ ]:
second_partial_step = 10 
second_partials = list(range(largest_first_partial, int(largest_total_rr), second_partial_step))

In [ ]:
columns = ['Earned RR', 'First Partial Value', 'First Partial %', 'Second Partial Value', 'Second Partial %']
rrs = pd.DataFrame(columns=columns)

In [ ]:
for fp in first_partials:
    for sp in second_partials:
        optimized = optimize.minimize(earned_rr_optimizer, [fp, 0.5, sp, 0.5], 
                              bounds=((0, largest_total_rr), (0, 1.), (0, largest_total_rr), (0, 1.)), method='trust-constr', constraints=cons)
        rrs = rrs.append(
            {
              'Earned RR': earned_rr(optimized.x), 
              'First Partial Value': optimized.x[0], 
              'First Partial %': optimized.x[1], 
              'Second Partial Value': optimized.x[2], 
              'Second Partial %': optimized.x[3]
             }, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


In [ ]:
# values may differ slighly per run based on how well the optimizer finds a minimum but it shouldn't matter if you round 
max_rr_row = rrs.iloc[rrs['Earned RR'].idxmax()]
print(f'''The most "Total RR" achieved is {max_rr_row['Earned RR']:.3f} by takning {max_rr_row['First Partial %']:.2f}% off at {max_rr_row['First Partial Value']:.3f}
      RR and then {max_rr_row['Second Partial %']:.2f}% off at {max_rr_row['Second Partial Value']:.3f} RR''')

The most total RR achieved is 29.154 by takning 0.68% off at 9.755
      RR and then 0.32% off at 90.059 RR


In [ ]:
rrs

,Earned RR,First Partial Value,First Partial %,Second Partial Value,Second Partial %
0,5.977528,2.987240,0.769807,10.552142,0.230193
1,7.502894,1.988197,0.745765,20.150555,0.254235
2,3.955773,0.684637,0.500004,30.022377,0.499996
3,13.708192,3.864126,0.736311,40.131438,0.263689
4,8.971979,0.687198,0.500003,50.020674,0.499997
5,13.529134,2.110024,0.747804,60.066700,0.252196
6,25.113447,9.773867,0.696266,70.420936,0.303734
7,17.509033,6.307489,0.698071,80.167481,0.301929
8,19.001072,2.086628,0.747892,89.928168,0.252108
9,8.592529,3.624279,0.696323,10.609927,0.303677
